#Load Data

In [1]:
!pip install matplotlib scipy opencv-python

In [ ]:
!apt-get update
!apt-get install unzip
!apt-get install ffmpeg libsm6 libxext6  -y

In [ ]:
!wget 'https://md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com/898720e7-9fcd-49f0-87ba-08c979e6f35e'
!mv '898720e7-9fcd-49f0-87ba-08c979e6f35e' 'ChestXrayData.zip'
!unzip ChestXrayData.zip

In [3]:
!ls

COVID19_Pneumonia_Normal_Chest_Xray_PA_Dataset	metadata.csv  onstart.sh
ChestXray.ipynb					models	      ports.log
ChestXrayData.zip				onstart.log   utils.py


In [4]:
import glob
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

In [5]:
class XrayDataset(Dataset):
    def __init__(self):
        self.imgs_path = "COVID19_Pneumonia_Normal_Chest_Xray_PA_Dataset/"
        file_list = glob.glob(self.imgs_path + "*")
        self.data = []
        for class_path in file_list:
            class_name = class_path.split("/")[-1]
            for img_path in glob.glob(class_path + "/*.jpeg"):
                self.data.append([img_path, class_name])
        self.class_map = {'covid':0, 'normal':1, 'pneumonia':2}
        self.classes = {'covid', 'normal','pneumonia'}
        self.img_dim = (32, 32)

    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        img_path, class_name = self.data[idx]
        img = cv2.imread(img_path)
        img = cv2.resize(img, self.img_dim)
        class_id = self.class_map[class_name]
        img_tensor = torch.from_numpy(img).float()
        img_tensor = img_tensor.permute(2, 0, 1)
        class_id = torch.tensor(class_id)
        return img_tensor, class_id

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision

from torchvision import *
from torch.utils.data import Dataset, DataLoader
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import time
import copy
import os

from models import *
from utils import progress_bar

batch_size = 16
learning_rate = 1e-3

transforms = transforms.Compose(
[
    transforms.ToTensor()
])

dataset = XrayDataset()

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

images, labels = next(iter(train_dataloader)) 
print("images-size:", images.shape)

out = torchvision.utils.make_grid(images)
print("out-size:", labels.shape)


images-size: torch.Size([16, 3, 32, 32])
out-size: torch.Size([16])


In [7]:
class EarlyStopping():
    def __init__(self, tolerance=5, min_delta=0):

        self.tolerance = tolerance
        self.min_delta = min_delta
        self.counter = 0
        self.early_stop = False

    def __call__(self, train_loss, validation_loss):
        if train_loss < self.min_delta:
            self.min_delta -= self.min_delta*0.5
            
        if (validation_loss - train_loss) > self.min_delta:
            self.counter +=1
            if self.counter >= self.tolerance:  
                self.early_stop = True
        else:
            self.counter = 0

# Training

In [8]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_dataloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        progress_bar(batch_idx, len(train_dataloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
    return train_loss


# Testing

In [9]:
def test(epoch,model_name):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_dataloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            progress_bar(batch_idx, len(test_dataloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/'+model_name+'.pth')
        best_acc = acc
    return test_loss


In [10]:
classes = dataset.classes
lr = 0.01
n_epochs = 20

In [11]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

net = DenseNet121()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

early_stopping = EarlyStopping(tolerance=5, min_delta=0.1)

epoch = 0
while(not(early_stopping.early_stop)):
    train_loss = train(epoch) / len(train_dataloader)
    validation_loss = test(epoch,'DenseNet') / len(test_dataloader)
    scheduler.step()
    early_stopping(train_loss,validation_loss)
    epoch+=1


Epoch: 0
 [================================================================>]  Step: 119ms | Tot: 9s731ms | Loss: 0.210 | Acc: 95.848% (1085/113 71/71 
 [=============================================================>...]  Step: 73ms | Tot: 1s671ms | Loss: 0.054 | Acc: 98.592% (280/28 18/18 
Saving..

Epoch: 1
 [================================================================>]  Step: 118ms | Tot: 9s842ms | Loss: 0.028 | Acc: 98.940% (1120/113 71/71 
 [=============================================================>...]  Step: 65ms | Tot: 1s609ms | Loss: 0.029 | Acc: 98.944% (281/28 18/18  
Saving..

Epoch: 2
 [================================================================>]  Step: 128ms | Tot: 9s541ms | Loss: 0.022 | Acc: 99.470% (1126/113 71/71 
 [=============================================================>...]  Step: 71ms | Tot: 1s616ms | Loss: 0.007 | Acc: 99.648% (283/28 18/18  
Saving..

Epoch: 3
 [================================================================>]  Step: 112ms 

In [12]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

net = PreActResNet18()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

early_stopping = EarlyStopping(tolerance=5, min_delta=0.1)

epoch = 0
while(not(early_stopping.early_stop)):
    train_loss = train(epoch) / len(train_dataloader)
    validation_loss = test(epoch,'PreActResNet') / len(test_dataloader)
    scheduler.step()
    early_stopping(train_loss,validation_loss)
    epoch+=1



Epoch: 0
 [================================================================>]  Step: 89ms | Tot: 6s981ms | Loss: 0.222 | Acc: 94.700% (1072/113 71/71  
 [=============================================================>...]  Step: 67ms | Tot: 1s449ms | Loss: 0.107 | Acc: 95.423% (271/28 18/18 
Saving..

Epoch: 1
 [================================================================>]  Step: 68ms | Tot: 6s973ms | Loss: 0.025 | Acc: 98.852% (1119/113 71/71  
 [=============================================================>...]  Step: 68ms | Tot: 1s444ms | Loss: 0.017 | Acc: 99.296% (282/28 18/18  
Saving..

Epoch: 2
 [================================================================>]  Step: 68ms | Tot: 6s942ms | Loss: 0.025 | Acc: 99.028% (1121/113 71/71  
 [=============================================================>...]  Step: 75ms | Tot: 1s429ms | Loss: 0.072 | Acc: 98.239% (279/28 18/18  

Epoch: 3
 [================================================================>]  Step: 102ms | Tot: 7s

In [ ]:
del net

In [13]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

net = DPN92()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

early_stopping = EarlyStopping(tolerance=5, min_delta=0.1)

epoch = 0
while(not(early_stopping.early_stop)):
    train_loss = train(epoch) / len(train_dataloader)
    validation_loss = test(epoch,'DPN') / len(test_dataloader)
    scheduler.step()
    early_stopping(train_loss,validation_loss)
    epoch+=1


Epoch: 0
 [================================================================>]  Step: 182ms | Tot: 16s445ms | Loss: 0.789 | Acc: 92.580% (1048/113 71/71 
 [=============================================================>...]  Step: 85ms | Tot: 1s876ms | Loss: 0.111 | Acc: 96.127% (273/28 18/18  ==================================>............................]  Step: 100ms | Tot: 1s90ms | Loss: 0.123 | Acc: 96.591% (170/17 11/18 
Saving..

Epoch: 1
 [================================================================>]  Step: 182ms | Tot: 16s441ms | Loss: 0.072 | Acc: 97.527% (1104/113 71/71 
 [=============================================================>...]  Step: 85ms | Tot: 1s836ms | Loss: 0.018 | Acc: 99.648% (283/28 18/18   
Saving..

Epoch: 2
 [================================================================>]  Step: 178ms | Tot: 16s388ms | Loss: 0.081 | Acc: 97.615% (1105/113 71/71 
 [=============================================================>...]  Step: 85ms | Tot: 1s823ms | Loss

In [ ]:
del net

In [14]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

net = EfficientNetB0()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

early_stopping = EarlyStopping(tolerance=5, min_delta=0.1)

epoch = 0
while(not(early_stopping.early_stop)):
    train_loss = train(epoch) / len(train_dataloader)
    validation_loss = test(epoch,'EffecientNet') / len(test_dataloader)
    scheduler.step()
    early_stopping(train_loss,validation_loss)
    epoch+=1


Epoch: 0
 [================================================================>]  Step: 88ms | Tot: 7s904ms | Loss: 0.986 | Acc: 81.095% (918/113 71/71  
 [=============================================================>...]  Step: 65ms | Tot: 1s517ms | Loss: 0.958 | Acc: 95.423% (271/28 18/18 
Saving..

Epoch: 1
 [================================================================>]  Step: 85ms | Tot: 7s967ms | Loss: 0.221 | Acc: 95.936% (1086/113 71/71  
 [=============================================================>...]  Step: 75ms | Tot: 1s563ms | Loss: 0.084 | Acc: 98.239% (279/28 18/18 
Saving..

Epoch: 2
 [================================================================>]  Step: 89ms | Tot: 7s987ms | Loss: 0.078 | Acc: 98.057% (1110/113 71/71  
 [=============================================================>...]  Step: 70ms | Tot: 1s548ms | Loss: 0.471 | Acc: 93.662% (266/28 18/18  

Epoch: 3
 [================================================================>]  Step: 107ms | Tot: 8s23

In [ ]:
del net

In [15]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

net = GoogLeNet()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

early_stopping = EarlyStopping(tolerance=5, min_delta=0.1)

epoch = 0
while(not(early_stopping.early_stop)):
    train_loss = train(epoch) / len(train_dataloader)
    validation_loss = test(epoch,'GoogleNet') / len(test_dataloader)
    scheduler.step()
    early_stopping(train_loss,validation_loss)
    epoch+=1


Epoch: 0
 [================================================================>]  Step: 115ms | Tot: 9s60ms | Loss: 0.183 | Acc: 95.053% (1076/113 71/71  
 [=============================================================>...]  Step: 67ms | Tot: 1s530ms | Loss: 0.061 | Acc: 97.887% (278/28 18/18  
Saving..

Epoch: 1
 [================================================================>]  Step: 101ms | Tot: 9s70ms | Loss: 0.031 | Acc: 98.940% (1120/113 71/71  
 [=============================================================>...]  Step: 70ms | Tot: 1s527ms | Loss: 0.015 | Acc: 99.648% (283/28 18/18  
Saving..

Epoch: 2
 [================================================================>]  Step: 88ms | Tot: 8s987ms | Loss: 0.039 | Acc: 98.675% (1117/113 71/71  
 [=============================================================>...]  Step: 77ms | Tot: 1s525ms | Loss: 0.022 | Acc: 98.592% (280/28 18/18  

Epoch: 3
 [================================================================>]  Step: 96ms | Tot: 9s

In [ ]:
del net

In [16]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

net = ResNeXt29_2x64d()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

early_stopping = EarlyStopping(tolerance=5, min_delta=0.1)

epoch = 0
while(not(early_stopping.early_stop)):
    train_loss = train(epoch) / len(train_dataloader)
    validation_loss = test(epoch,'ResNext') / len(test_dataloader)
    scheduler.step()
    early_stopping(train_loss,validation_loss)
    epoch+=1


Epoch: 0
 [================================================================>]  Step: 110ms | Tot: 10s145ms | Loss: 0.338 | Acc: 93.816% (1062/113 71/71 
 [=============================================================>...]  Step: 78ms | Tot: 1s740ms | Loss: 0.047 | Acc: 97.535% (277/28 18/18  
Saving..

Epoch: 1
 [================================================================>]  Step: 130ms | Tot: 10s97ms | Loss: 0.049 | Acc: 98.322% (1113/113 71/71 
 [=============================================================>...]  Step: 89ms | Tot: 1s778ms | Loss: 0.034 | Acc: 98.944% (281/28 18/18  
Saving..

Epoch: 2
 [================================================================>]  Step: 125ms | Tot: 10s131ms | Loss: 0.060 | Acc: 97.261% (1101/113 71/71 
 [=============================================================>...]  Step: 104ms | Tot: 1s723ms | Loss: 0.058 | Acc: 98.239% (279/28 18/18 

Epoch: 3
 [================================================================>]  Step: 100ms | Tot:

In [ ]:
del net

In [17]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

net = DLA()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

early_stopping = EarlyStopping(tolerance=5, min_delta=0.1)

epoch = 0
while(not(early_stopping.early_stop)):
    train_loss = train(epoch) / len(train_dataloader)
    validation_loss = test(epoch,'DLA') / len(test_dataloader)
    scheduler.step()
    early_stopping(train_loss,validation_loss)
    epoch+=1


Epoch: 0
 [================================================================>]  Step: 103ms | Tot: 7s827ms | Loss: 0.226 | Acc: 94.435% (1069/113 71/71 
 [=============================================================>...]  Step: 56ms | Tot: 1s481ms | Loss: 0.300 | Acc: 97.535% (277/28 18/18 
Saving..

Epoch: 1
 [================================================================>]  Step: 81ms | Tot: 7s836ms | Loss: 0.077 | Acc: 96.820% (1096/113 71/71  
 [=============================================================>...]  Step: 76ms | Tot: 1s467ms | Loss: 0.026 | Acc: 99.296% (282/28 18/18  
Saving..

Epoch: 2
 [================================================================>]  Step: 102ms | Tot: 7s813ms | Loss: 0.046 | Acc: 98.322% (1113/113 71/71 
 [=============================================================>...]  Step: 61ms | Tot: 1s520ms | Loss: 0.077 | Acc: 97.887% (278/28 18/18 

Epoch: 3
 [================================================================>]  Step: 93ms | Tot: 7s70